<a href="https://colab.research.google.com/github/Antsruin/GNN-HLS-FPGA/blob/main/Simple_GNN_to_hls_codes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!!pip install hls4ml
# Upload gnn_fpga_ready.h5
# Run the conversion script

['Collecting hls4ml',
 '  Downloading hls4ml-1.2.0-py3-none-any.whl.metadata (11 kB)',
 'Requirement already satisfied: h5py in /usr/local/lib/python3.12/dist-packages (from hls4ml) (3.15.1)',
 'Requirement already satisfied: numpy in /usr/local/lib/python3.12/dist-packages (from hls4ml) (2.0.2)',
 'Collecting pydigitalwavetools==1.1 (from hls4ml)',
 '  Downloading pyDigitalWaveTools-1.1-py3-none-any.whl.metadata (3.1 kB)',
 'Requirement already satisfied: pyyaml in /usr/local/lib/python3.12/dist-packages (from hls4ml) (6.0.3)',
 'Collecting quantizers (from hls4ml)',
 '  Downloading quantizers-1.2.2-py3-none-any.whl.metadata (4.7 kB)',
 'Downloading hls4ml-1.2.0-py3-none-any.whl (3.2 MB)',
 '\x1b[?25l   \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/3.2 MB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K   \x1b━━━━━━━━━━━━\x1b\x1b╸\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b1.0/3.2 MB\x1b \x1b35.3 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K   \x1b━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╸

In [8]:
!pip install vivado_hls

ERROR: Could not find a version that satisfies the requirement vivado_hls (from versions: none)
ERROR: No matching distribution found for vivado_hls


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import hls4ml

In [3]:
# Custom GNN Layer using Keras
class GNNLayer(layers.Layer):
    """
    Simple Graph Neural Network Layer
    Performs: Aggregate(neighbors) -> Dense -> Activation
    """
    def __init__(self, units, activation='relu', **kwargs):
        super(GNNLayer, self).__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, input_shape):
        # input_shape: [(batch, nodes, features), (batch, nodes, nodes)]
        feature_dim = input_shape[0][-1]
        self.dense = layers.Dense(self.units, use_bias=True)
        super(GNNLayer, self).build(input_shape)

    def call(self, inputs):
        node_features, adj_matrix = inputs

        # Aggregate: Matrix multiply adjacency with features
        # adj_matrix: (batch, nodes, nodes)
        # node_features: (batch, nodes, features)
        aggregated = tf.matmul(adj_matrix, node_features)

        # Transform with dense layer
        output = self.dense(aggregated)

        # Apply activation
        output = self.activation(output)

        return output

    def get_config(self):
        config = super(GNNLayer, self).get_config()
        config.update({
            'units': self.units,
            'activation': keras.activations.serialize(self.activation)
        })
        return config

In [4]:
# Build a simple GNN model
def create_gnn_model(num_nodes=4, input_features=3, hidden_dim=8, output_dim=2):
    """
    Create a simple 2-layer GNN model
    """
    # Input layers
    node_features_input = keras.Input(shape=(num_nodes, input_features), name='node_features')
    adj_matrix_input = keras.Input(shape=(num_nodes, num_nodes), name='adj_matrix')

    # Layer 1: GNN with ReLU
    h1 = GNNLayer(hidden_dim, activation='relu', name='gnn_layer1')(
        [node_features_input, adj_matrix_input]
    )

    # Layer 2: GNN without activation (for output)
    h2 = GNNLayer(output_dim, activation='linear', name='gnn_layer2')(
        [h1, adj_matrix_input]
    )

    model = keras.Model(
        inputs=[node_features_input, adj_matrix_input],
        outputs=h2,
        name='simple_gnn'
    )

    return model


In [5]:
# Alternative: Simplified version using only Dense layers (better for HLS4ML)
def create_simplified_gnn(num_nodes=4, input_features=3, hidden_dim=8, output_dim=2):
    """
    Simplified GNN using only standard Keras layers for better HLS4ML compatibility
    Flattens the graph structure for easier synthesis
    """
    # Flatten approach: concatenate all node features after aggregation
    input_features_flat = keras.Input(shape=(num_nodes * input_features,), name='features_flat')

    # Hidden layer 1
    h1 = layers.Dense(hidden_dim * num_nodes, activation='relu', name='dense1')(input_features_flat)

    # Hidden layer 2
    h2 = layers.Dense(hidden_dim * num_nodes, activation='relu', name='dense2')(h1)

    # Output layer
    output = layers.Dense(num_nodes * output_dim, activation='linear', name='output')(h2)

    model = keras.Model(inputs=input_features_flat, outputs=output, name='simplified_gnn')

    return model


In [6]:
# HLS4ML Conversion Function
def convert_to_fpga(model, output_dir='gnn_hls4ml', board='pynq-z2'):
    """
    Convert Keras model to FPGA using HLS4ML

    Parameters:
    - model: Keras model to convert
    - output_dir: Directory for HLS4ML output
    - board: Target FPGA board (pynq-z2, zcu102, etc.)
    """

    # Configure HLS4ML
    config = hls4ml.utils.config_from_keras_model(model, granularity='name')

    # Set precision for weights and activations
    config['Model']['Precision'] = 'ap_fixed<16,6>'
    config['Model']['ReuseFactor'] = 1

    # Configure each layer
    for layer in config['LayerName'].keys():
        config['LayerName'][layer]['Precision'] = {
            'weight': 'ap_fixed<16,6>',
            'bias': 'ap_fixed<16,6>',
            'result': 'ap_fixed<16,6>'
        }

    # HLS config
    hls_config = {
        'Model': {
            'Precision': 'ap_fixed<16,6>',
            'ReuseFactor': 1,
            'Strategy': 'Latency',  # or 'Resource'
            'BramFactor': 10000,
        }
    }

    print("Converting model to HLS...")
    print(f"Output directory: {output_dir}")

    # Convert to HLS
    hls_model = hls4ml.converters.convert_from_keras_model(
        model,
        hls_config=hls_config,
        output_dir=output_dir,
        part=f'xc7z020clg400-1' if board == 'pynq-z2' else 'xczu9eg-ffvb1156-2-e',
        clock_period=5,
        io_type='io_parallel'  # or 'io_stream'
    )

    # Compile the HLS model
    hls_model.compile()

    print("HLS model compiled successfully!")

    return hls_model

In [9]:
# Example usage and testing
if __name__ == "__main__":
    print("=" * 60)
    print("GNN to FPGA with HLS4ML")
    print("=" * 60)

    # Parameters
    num_nodes = 4
    input_features = 3
    hidden_dim = 8
    output_dim = 2

    # Create sample data
    # Adjacency matrix (with self-loops, normalized)
    adj = np.array([
        [1, 1, 0, 0],
        [1, 1, 1, 0],
        [0, 1, 1, 1],
        [0, 0, 1, 1]
    ], dtype=np.float32)

    # Normalize adjacency matrix
    degree = np.sum(adj, axis=1, keepdims=True)
    adj_normalized = adj / degree
    adj_normalized = adj_normalized[np.newaxis, ...]  # Add batch dimension

    # Node features
    features = np.array([
        [1.0, 0.5, 0.2],
        [0.8, 0.3, 0.9],
        [0.2, 0.7, 0.4],
        [0.6, 0.1, 0.8]
    ], dtype=np.float32)
    features = features[np.newaxis, ...]  # Add batch dimension

    print("\n1. Creating Simplified GNN Model (Best for HLS4ML)...")
    # Use simplified version - better compatibility
    features_flat = features.reshape(1, -1)
    model = create_simplified_gnn(num_nodes, input_features, hidden_dim, output_dim)
    model.compile(optimizer='adam', loss='mse')

    print("\nModel Summary:")
    model.summary()

    # Test the model
    print("\n2. Testing Model...")
    output = model.predict(features_flat, verbose=0)
    print(f"Input shape: {features_flat.shape}")
    print(f"Output shape: {output.shape}")
    print(f"Sample output:\n{output.reshape(num_nodes, output_dim)}")

    # Convert to FPGA
    print("\n3. Converting to FPGA with HLS4ML...")
    print("Note: This requires hls4ml to be installed: pip install hls4ml")
    print("      And Xilinx Vivado HLS to be available on the system")


    hls_model = convert_to_fpga(
        model,
        output_dir='gnn_fpga_output',
        board='pynq-z2'
    )

    # Test HLS model
    print("\n4. Testing HLS Model...")
    hls_output = hls_model.predict(features_flat)
    print(f"HLS Output shape: {hls_output.shape}")
    print(f"HLS Output:\n{hls_output.reshape(num_nodes, output_dim)}")

    # Compare outputs
    print("\n5. Comparing Keras vs HLS outputs...")
    diff = np.abs(output - hls_output)
    print(f"Maximum difference: {np.max(diff):.6f}")
    print(f"Mean difference: {np.mean(diff):.6f}")


    # Save the Keras model for later use
    print("\n7. Saving Keras model...")
    model.save('gnn_model.h5')
    print("Model saved as 'gnn_model.h5'")
    print("\nYou can load it later with: model = keras.models.load_model('gnn_model.h5')")

    print("\n" + "=" * 60)
    print("Process Complete!")
    print("=" * 60)

GNN to FPGA with HLS4ML

1. Creating Simplified GNN Model (Best for HLS4ML)...

Model Summary:


Model: "simplified_gnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ features_flat (InputLayer)      │ (None, 12)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ (None, 32)             │           416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense2 (Dense)                  │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 8)              │           264 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,736 (6.78 KB)

 Trainable params: 1,736 (6.78 KB)

 Non-trainable params: 0 (0.00 B)


2. Testing Model...
Input shape: (1, 12)
Output shape: (1, 8)
Sample output:
[[ 0.1494176   0.03711023]
 [ 0.0198466   0.36051148]
 [ 0.09396695  0.43737856]
 [-0.36304754  0.03990293]]

3. Converting to FPGA with HLS4ML...
Note: This requires hls4ml to be installed: pip install hls4ml
      And Xilinx Vivado HLS to be available on the system
Converting model to HLS...
Output directory: gnn_fpga_output


HLS model compiled successfully!

4. Testing HLS Model...
HLS Output shape: (8,)
HLS Output:
[[ 0.12988281  0.02929688]
 [ 0.02246094  0.3251953 ]
 [ 0.08496094  0.41210938]
 [-0.35839844  0.02734375]]

5. Comparing Keras vs HLS outputs...
Maximum difference: 0.035316
Mean difference: 0.014595

7. Saving Keras model...
Model saved as 'gnn_model.h5'

You can load it later with: model = keras.models.load_model('gnn_model.h5')

Process Complete!
